In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
import os

In [2]:
files = os.listdir("C:\\Users\\kevin\\ziknet-trends-rolling\\data\\Mexico\\processed_data\\")
weeks_ahead = 3

In [3]:
datasets = {}
for file in files:
    datasets[file] = pd.read_csv("C:\\Users\\kevin\\ziknet-trends-rolling\\data\\Mexico\\processed_data\\{}".format(file))
    print(file)
    print(datasets[file].head())
    print("Number of observations: ", len(datasets[file]))

Chiapas_2016-2017.csv
         Date  Searches  Cases
0  02/01/2016       100      9
1  09/01/2016       100      0
2  16/01/2016       100      3
3  23/01/2016        55     11
4  30/01/2016        98     11
Number of observations:  104
Guerrero_2016-2017.csv
         Date  Searches  Cases
0  02/01/2016         0      0
1  09/01/2016        26      0
2  16/01/2016        32      0
3  23/01/2016        70      0
4  30/01/2016        75      1
Number of observations:  104
NuevoLeon_2016-2017.csv
         Date  Searches  Cases
0  02/01/2016         4      3
1  09/01/2016         2      0
2  16/01/2016         9      0
3  23/01/2016        36      0
4  30/01/2016        67      0
Number of observations:  104
Veracruz_2016-2017.csv
         Date  Searches  Cases
0  02/01/2016         8      0
1  09/01/2016         0      0
2  16/01/2016        23      0
3  23/01/2016        46      0
4  30/01/2016        68      1
Number of observations:  104
Yucatan_2016-2017.csv
         Date  Searches  C

In [4]:
def series_to_supervised(df, outputColumn, n_in=1, n_out=1, dropnan=True):
    n_vars = df.shape[1]
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [("{}(t-{})".format(col, i)) for col in df.columns]

    # Append next observation[outputColumn] at n_out obs
    cols.append(df[outputColumn].shift(-n_out+1))
    names+=[outputColumn + "(t+{})".format(n_out-1)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [5]:
def getXY(dataset, state, weeksAhead):
    n_features = dataset.shape[1]
    
    n_weeks = 4
    reframed = series_to_supervised(dataset, "Cases",  n_weeks, weeksAhead)
    values = reframed.values
    
    totalFeatures = values.shape[1]

    x,y = values[:, :totalFeatures-1], values[:, totalFeatures-1] #Y is the last column, X is all the previous columns 

    x = x.reshape((x.shape[0], n_weeks, n_features)) # Reshape as 3-D
    return x, y

In [6]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Flatten, Dense, LSTM, BatchNormalization
from keras.optimizers import Adam
from keras.layers.merge import concatenate
from keras.constraints import non_neg
def LSTM_NN_Model():
    input_layer = Input(shape=(4,2))
    
    
    b1_out = LSTM(16, return_sequences=False)(input_layer)

    b2_out = Dense(128, activation="relu", kernel_regularizer="l2")(input_layer)
    b2_out = Flatten()(b2_out)

    concatenated = concatenate([b1_out, b2_out])
    
    out = Dense(8, activation="relu", kernel_regularizer="l2")(concatenated)
    out = Dense(8, activation="relu", kernel_regularizer="l2")(out)
#     out = Dense(1, activation="linear", kernel_constraint=non_neg(), name='output_layer')(out)
    out = Dense(1, activation="linear", name='output_layer')(out)

    model = Model([input_layer], out)
    model.compile(loss=["mse"], optimizer=Adam(0.0001), metrics=["mae"])

    return model

Using TensorFlow backend.


In [7]:
for file in datasets:
    dataset = datasets[file]
    x, y = getXY(dataset[["Searches", "Cases"]], "", weeks_ahead)
    model = LSTM_NN_Model()
    
    xDim1 = x[0].shape[0]
    xDim2 = x[0].shape[1]
    
    splitIndex = len(x) - 52

    train_X = x[:splitIndex]
    train_y = y[:splitIndex]

    test_y = y[splitIndex:]
    outDataset = pd.DataFrame()
    outDataset["Observerd"] = test_y

    model.fit(
        train_X,
        train_y,
        epochs=200,
        batch_size=4,
        verbose=0,
        shuffle=False)

    predicted_y_history = []

    while(splitIndex < len(y)):
        predicted_y = \
            model.predict(x[splitIndex].reshape(1, xDim1, xDim2))[0]

        model.fit(
            x[:splitIndex+1],
            y[:splitIndex+1],
            epochs=40,
            batch_size=4,
            verbose=0,
            shuffle=False)

        predicted_y_history.append(predicted_y[0])
        splitIndex += 1

    outDataset["PREDICTED"] = predicted_y_history
    print("{} RMSE: {}".format(file, mean_squared_error(test_y, predicted_y_history)))
    outDataset.to_csv(file)

Chiapas_2016-2017.csv RMSE: 12.187787185483607
Guerrero_2016-2017.csv RMSE: 25.126471995125183
NuevoLeon_2016-2017.csv RMSE: 990.2055142379425


KeyboardInterrupt: 